## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-12-20-57-00 +0000,wsj,The highly classified whistleblower complaint ...,https://www.wsj.com/politics/national-security...
1,2026-02-12-20-56-27 +0000,bbc,"Choi, 17, denies Kim historic third halfpipe gold",https://www.bbc.com/sport/articles/cly2jknym20...
2,2026-02-12-20-56-09 +0000,startribune,Trump's EPA revokes scientific finding that un...,https://www.startribune.com/trumps-epa-revokes...
3,2026-02-12-20-52-24 +0000,nyt,"From the chaos of snowboard cross, a repeat ch...",https://www.nytimes.com/2026/02/12/world/from-...
4,2026-02-12-20-51-53 +0000,nyt,DHS Shutdown Nears as Immigration Enforcement ...,https://www.nytimes.com/2026/02/12/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2500/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
27,trump,47
91,he,15
45,immigration,14
305,epstein,14
125,new,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
307,2026-02-12-00-44-00 +0000,wsj,The GOP-led House passed a resolution designed...,https://www.wsj.com/politics/policy/gop-led-ho...,122
181,2026-02-12-15-00-00 +0000,wsj,Josh Gruenbaum’s brash ways as a federal agenc...,https://www.wsj.com/politics/national-security...,98
98,2026-02-12-18-11-00 +0000,wsj,Trump Administration to End Immigration Enforc...,https://www.wsj.com/politics/policy/tom-homan-...,96
321,2026-02-11-23-37-50 +0000,nypost,Six House Republicans break with White House t...,https://nypost.com/2026/02/11/us-news/six-hous...,96
40,2026-02-12-20-01-49 +0000,nyt,Trump Administration to End Surge of Immigrati...,https://www.nytimes.com/2026/02/12/us/minnesot...,93


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
307,122,2026-02-12-00-44-00 +0000,wsj,The GOP-led House passed a resolution designed...,https://www.wsj.com/politics/policy/gop-led-ho...
199,62,2026-02-12-14-00-00 +0000,latimes,He was flying from L.A. to Houston. So how did...,https://www.latimes.com/california/story/2026-...
5,60,2026-02-12-20-51-00 +0000,wsj,The Department of Homeland Security is facing ...,https://www.wsj.com/politics/policy/dhs-fundin...
354,53,2026-02-11-21-42-00 +0000,wsj,"At a congressional hearing, Attorney General P...",https://www.wsj.com/politics/policy/pam-bondi-...
100,50,2026-02-12-18-09-20 +0000,nypost,"Pentagon can’t dock Sen. Mark Kelly’s rank, pa...",https://nypost.com/2026/02/12/us-news/pentagon...
313,46,2026-02-12-00-01-41 +0000,nypost,Mother killed by transgender Canadian school s...,https://nypost.com/2026/02/11/world-news/mothe...
85,44,2026-02-12-18-37-02 +0000,nypost,Inside New York City’s brand-new ‘bathing fest...,https://nypost.com/2026/02/12/health/inside-ny...
0,42,2026-02-12-20-57-00 +0000,wsj,The highly classified whistleblower complaint ...,https://www.wsj.com/politics/national-security...
297,42,2026-02-12-02-02-00 +0000,wsj,Demand for healthcare workers is outstripping ...,https://www.wsj.com/economy/jobs/healthcare-jo...
57,41,2026-02-12-19-35-05 +0000,nyt,Tumbler Ridge Shooting: What We Know About the...,https://www.nytimes.com/2026/02/11/world/ameri...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
